# trace

In [ ]:
//// test

open testing

## types

In [ ]:
inl types () =
    env.types ()
    guid.types ()
    rust.types ()
    sm'.types ()

## trace

### trace_level

In [ ]:
union trace_level =
    | Verbose
    | Debug
    | Info
    | Warning
    | Critical

### read_state

In [ ]:
inl read_state () =
    run_target function
        | Rust (Contract | Wasm) => fun () =>
            {
                trace_level = None
                repl_start = None
            }
        | _ => fun () =>
            {
                trace_level =
                    (join "TRACE_LEVEL")
                    |> env.get_environment_variable
                    |> reflection.union_try_pick
                repl_start =
                    inl automation = env.get_environment_variable (join "AUTOMATION")
                    if automation = "True"
                    then date_time.now () |> date_time.ticks |> fun (date_time.timestamp x) => x |> Some
                    else None
            }

### trace_state

In [ ]:
type trace_state =
    {
        count : mut i64
        trace_file : mut (string -> ())
        enabled : mut bool
        level : mut trace_level
        repl_start : optionm'.option' i64
    }

inl new_trace_state trace_level' =
    inl { repl_start trace_level } = read_state ()
    {
        enabled = mut true
        count = mut 0i64
        level = trace_level |> optionm'.default_value trace_level' |> mut
        trace_file = mut ignore
        repl_start = repl_start |> optionm'.box
    } : trace_state

### init_trace_state

In [ ]:
inl init_trace_state trace_level : () =
    inl trace_level =
        trace_level
        |> optionm'.default_value Verbose
    backend_switch {
        Fsharp = fun () =>
            global "module State = let mutable trace_state = None"
            $'if State.trace_state.IsNone then State.trace_state <- !new_trace_state !trace_level |> Some' : ()
        Python = fun () =>
            global "class State: trace_state = None"
            $'if State.trace_state is None: State.trace_state = !new_trace_state(!trace_level)' : ()
    }

### get_trace_state_or_init

In [ ]:
inl get_trace_state_or_init trace_level : trace_state =
    init_trace_state trace_level
    // $'match State.trace_state with Some x -> x | None -> failwith "trace.get_trace_state_or_init / State.trace_state=None"'
    backend_switch {
        Fsharp = fun () =>
            $'State.trace_state.Value' : trace_state
        Python = fun () =>
            $'State.trace_state' : trace_state
    }

### _locals

In [ ]:
inl _locals () = ""

### test_trace_level

In [ ]:
inl test_trace_level level : bool =
    inl state = get_trace_state_or_init None
    inl level' = *state.level
    if *state.enabled |> not
    then false
    else
        inl level : i32 = real real_core.union_tag level
        inl level' : i32 = real real_core.union_tag level'
        level >= level'

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

test_trace_level Critical |> _assert_eq true
test_trace_level Verbose |> _assert_eq true

inl level = get_trace_state_or_init None .level
level <- Debug
test_trace_level Verbose |> _assert_eq false
level <- Verbose
test_trace_level Verbose |> _assert_eq true

.py output (Cuda):
assert_eq / actual: True / expected: True
assert_eq / actual: True / expected: True
assert_eq / actual: False / expected: False
assert_eq / actual: True / expected: True

.rs output:
assert_eq / actual: true / expected: true
assert_eq / actual: true / expected: true
assert_eq / actual: false / expected: false
assert_eq / actual: true / expected: true

.ts output:
assert_eq / actual: true / expected: true
assert_eq / actual: true / expected: true
assert_eq / actual: false / expected: false
assert_eq / actual: true / expected: true

.py output:
assert_eq / actual: true / expected: true
assert_eq / actual: true / expected: true
assert_eq / actual: false / expected: false
assert_eq / actual: true / expected: true


.fsx output:
assert_eq / actual: true / expected: true
assert_eq / actual: true / expected: true
assert_eq / actual: false / expected: false
assert_eq / actual: true / expected: true


### trace_raw

In [ ]:
let rec trace_raw level fn =
    inl trace_state = get_trace_state_or_init None
    if level |> test_trace_level then
        inl count = trace_state.count
        count <- *count + 1

        inl text =
            backend_switch {
                Fsharp = fun () => $'$"%s{!fn ()}"' : string
                Python = fun () => fn () : string
            }
        run_target function
            | Rust _ => fun () =>
                open rust_operators
                !\\(text, $'\@"println\!(""{}"", $0)"')
            | _ => fun () =>
                backend_switch {
                    Fsharp = fun () => $'System.Console.WriteLine !text ' : ()
                    Python = fun () => $'print(!text)' : ()
                }

        *trace_state.trace_file text

### trace

In [ ]:
let trace (level : trace_level) (text_fn : () -> string) (locals : () -> string) =
    fun () =>
        inl trace_state = get_trace_state_or_init None
        inl time =
            run_target fun target =>
                match target with
                | Rust (Contract) => fun () => join ""
                | _ => fun () =>
                    match trace_state.repl_start |> optionm'.unbox with
                    | Some repl_start =>
                        inl t =
                            (date_time.now () |> date_time.ticks |> fun (date_time.timestamp x) => x)
                            - repl_start |> date_time.time_span
                        date_time.date_time_milliseconds
                            1i32 1i32 1i32
                            (t |> date_time.hours)
                            (t |> date_time.minutes)
                            (t |> date_time.seconds)
                            (t |> date_time.milliseconds)
                    | None => date_time.now ()
                    |> date_time.format (
                        backend_switch {
                            Fsharp = fun () =>
                                match target with
                                | Rust _ => join "hh:mm:ss"
                                | _ => join "HH:mm:ss"
                            Python = fun () => "%H:%M:%S"
                        }
                    )
        inl level_str = level |> reflection.union_to_string |> sm'.to_lower |> sm'.pad_left 7 ' '
        inl level_str =
            run_target function
            | Rust _ => fun () =>
                open rust_operators
                inl color : rust.ref' sm'.str =
                    match level with
                    | Verbose => !\($'"inline_colorization::color_bright_black"')
                    | Debug => !\($'"inline_colorization::color_bright_blue"')
                    | Info => !\($'"inline_colorization::color_bright_green"')
                    | Warning => !\($'"inline_colorization::color_yellow"')
                    | Critical => !\($'"inline_colorization::color_bright_red"')
                inl level_str = level_str |> sm'.as_str
                inl color_reset : rust.ref' sm'.str = !\($'"inline_colorization::color_reset"')
                $'"\\\"{!color}{!level_str}{!color_reset}\\\""'
                |> sm'.format''
                |> sm'.from_std_string
            | _ => fun () =>
                inl color =
                    match level with
                    | Verbose => $'"\\u001b[90m"'
                    | Debug => $'"\\u001b[94m"'
                    | Info => $'"\\u001b[92m"'
                    | Warning => $'"\\u001b[93m"'
                    | Critical => $'"\\u001b[91m"'
                inl color_reset = join $'"\\u001b[0m"'
                color +. level_str +. color_reset
        inl count = *trace_state.count
        backend_switch {
            Fsharp = fun () => $'$"{!time} {!level_str} #{!count} %s{!text_fn ()} / %s{!locals ()}"' : string
            Python = fun () => $'f"{!time} {!level_str} #{!count} {!text_fn()} / {!locals()}"' : string
        }
        |> sm'.trim_start []
        |> sm'.trim_end [ ' '; '/' ]
    |> trace_raw level

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

types ()
trace Debug (fun () => "test1") _locals
trace Debug (fun () => "test2") _locals
get_trace_state_or_init None .count
|> fun x => *x
|> _assert_eq 2

.py output (Cuda):
00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2 / expected: 2

.rs output:
00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2 / expected: 2

.ts output:
00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2 / expected: 2

.py output:
00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2 / expected: 2


.fsx output:
00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2L / expected: 2L


## main

In [ ]:
inl main () =
    types ()
    init_trace_state None
    $'let trace x = !trace x' : ()